<a href="https://colab.research.google.com/github/marcosandrey85/EDA_gorjertas/blob/main/Bandas_de_Bollinger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando as bibliotecas

In [ ]:
!pip install yfinance 
import yfinance as yf
import pandas as pd
import numpy as np


# Ação que queremos trabalhar

In [ ]:
acao = yf.Ticker('OIBR3.SA')
data = acao.history(period ="1y" )
df = data[['Close']]
df

,Close
Date,
2021-02-04,2.16
2021-02-05,2.12
2021-02-08,2.11
2021-02-09,2.11
2021-02-10,2.04
...,...
2022-01-31,1.07
2022-02-01,1.06
2022-02-02,1.04


# Calculos estatiticos 

### Média móvel

In [ ]:
mm = df.rolling(window = 20).mean()
mm

,Close
Date,
2021-02-04,NaN
2021-02-05,NaN
2021-02-08,NaN
2021-02-09,NaN
2021-02-10,NaN
...,...
2022-01-31,0.8215
2022-02-01,0.8385
2022-02-02,0.8550


### Desvio padrão

In [ ]:
dpm = df.rolling(window=20).std()
dpm

,Close
Date,
2021-02-04,NaN
2021-02-05,NaN
2021-02-08,NaN
2021-02-09,NaN
2021-02-10,NaN
...,...
2022-01-31,0.125668
2022-02-01,0.133939
2022-02-02,0.137554


### Calculo da Banda superior e inferior

In [ ]:
sup_band = mm + (0.07*dpm)
inf_band = mm - (0.07*dpm)

**Alterando o nome das colunas de banda sup e banda inf**

In [ ]:
sup_band= sup_band.rename(columns = {'Close' : 'superior'})
inf_band= inf_band.rename(columns = {'Close' : 'inferior'})

# Unir todas as colunas

In [ ]:
bandas_bollinger = df.join(sup_band).join(inf_band)
bandas_bollinger

,Close,superior,inferior
Date,,,
2021-02-04,2.16,NaN,NaN
2021-02-05,2.12,NaN,NaN
2021-02-08,2.11,NaN,NaN
2021-02-09,2.11,NaN,NaN
2021-02-10,2.04,NaN,NaN
...,...,...,...
2022-01-31,1.07,0.830297,0.812703
2022-02-01,1.06,0.847876,0.829124
2022-02-02,1.04,0.864629,0.845371


### Retirar os nulos

In [ ]:
bandas_bollinger = bandas_bollinger.dropna()


# Calculo dos pontos de compra e venda

In [ ]:
compra = bandas_bollinger[bandas_bollinger['Close'] <= bandas_bollinger['inferior']]
venda =  bandas_bollinger[bandas_bollinger['Close'] >= bandas_bollinger['superior']]

# Plotando o Gráfico

In [ ]:
import plotly.io as pio 
import plotly.graph_objects as go

pio.templates.default = "plotly_dark"

fig = go.Figure()
fig.add_trace(go.Scatter(x = inf_band.index,
                         y = inf_band['inferior'],
                         name = 'Banda Inferior',
                         line_color = 'rgba(173,204,255,0.2)'
                         ))
fig.add_trace(go.Scatter(x = sup_band.index,
                         y = sup_band['superior'],
                         name = 'Banda Superior',
                         fill = 'tonexty',
                         fillcolor = 'rgba(173,204,255,0.2)',
                         line_color = 'rgba(173,204,255,0.2)'
                         ))
fig.add_trace(go.Scatter(x = df.index,
                         y = df['Close'],
                         name = 'Preço fechamento',
                         line_color = '#636EFA'
                         ))
fig.add_trace(go.Scatter(x = mm.index,
                         y = mm['Close'],
                         name = 'Média móvel',
                         line_color = '#FECB52'
                         ))
fig.add_trace(go.Scatter(x = compra.index,
                         y = compra['Close'],
                         name = 'Compra',
                         mode = 'markers',
                         marker = dict(
                             color = '#00CC96',
                             size = 8
                             )
                         ))
fig.add_trace(go.Scatter(x = venda.index,
                         y = venda['Close'],
                         name = 'Venda',
                         mode = 'markers',
                         marker = dict(
                             color = '#EF553B',
                             size = 8
                             )
                        ))
fig.show()

In [ ]:
bandas_bollinger

,Close,superior,inferior
Date,,,
2021-03-05,1.99,1.980973,1.964027
2021-03-08,1.79,1.962340,1.945660
2021-03-09,1.75,1.943950,1.927050
2021-03-10,1.75,1.925912,1.909088
2021-03-11,1.92,1.915794,1.900206
...,...,...,...
2022-01-31,1.07,0.830297,0.812703
2022-02-01,1.06,0.847876,0.829124
2022-02-02,1.04,0.864629,0.845371


In [ ]:
import plotly
